In [1]:
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

Using TensorFlow backend.


In [2]:
TRAIN_DATA_DIR='./road_segmentation/training/'
TEST_DATA_DIR='./road_segmentation/test_set_images/'
NUM_TRAIN_FILES = 100
NUM_TEST_FILES = 50
IMG_SIZE = 400
TEST_IMG_SIZE = 608
PATCH_INPUT_SIZE = 32
PATCH_SIZE = 16
NUM_CLASSES = 2
RANDOM_SEED = 1337
BATCH_SIZE = 512

In [3]:
# Load data
X = np.zeros((NUM_TRAIN_FILES, IMG_SIZE, IMG_SIZE, 3))
for i in range(1, NUM_TRAIN_FILES+1):
    X[i-1] = mpimg.imread(TRAIN_DATA_DIR + "images/satImage_%.3d.png" % i)

Y = np.zeros((NUM_TRAIN_FILES, IMG_SIZE, IMG_SIZE))
for i in range(1, NUM_TRAIN_FILES+1):
    Y[i-1] = mpimg.imread(TRAIN_DATA_DIR + "groundtruth/satImage_%.3d.png" % i)
    
X_test = np.zeros((NUM_TEST_FILES, TEST_IMG_SIZE, TEST_IMG_SIZE, 3))
for i in range(1, NUM_TEST_FILES+1):
    X_test[i-1] = mpimg.imread(TEST_DATA_DIR + "test_%d/test_%d.png" % (i, i))

# Quantize labels
#Y = (Y > 0).astype(np.int)
        
def prep_data(X, Y, img_size, train):    
    # Cut into patches
    patches_per_axis = img_size // PATCH_SIZE
    
    # Padding
    padding = (PATCH_INPUT_SIZE - PATCH_SIZE) // 2
    X_padded = np.zeros(X.shape + np.array([0, 2*padding, 2*padding, 0]))
    X_padded[:, padding:-padding, padding:-padding, :] = X
    
    if train:
        step = PATCH_SIZE // 2
        num_samples = X.shape[0] * (patches_per_axis**2) * 4
    else:
        step = PATCH_SIZE
        num_samples = X.shape[0] * (patches_per_axis**2)
    
    X_patches = np.zeros((num_samples, PATCH_INPUT_SIZE, PATCH_INPUT_SIZE, X.shape[3]))
    Y_patches = np.zeros(num_samples)
    
    idx = 0
    for i in tqdm(range(X.shape[0])):
        for x in range(padding, img_size, step):
            for y in range(padding, img_size, step):
                X_patches[idx] = X_padded[i, x-padding:x+PATCH_SIZE+padding, y-padding:y+PATCH_SIZE+padding, :]
                if Y is not None:
                    Y_patches[idx] = Y[i, x:x+PATCH_SIZE, y:y+PATCH_SIZE].mean()
                idx += 1
            
    if Y is not None:
#         Y = Y.reshape((-1, patches_per_axis, PATCH_SIZE, patches_per_axis, PATCH_SIZE)) \
#             .transpose((0, 1, 3, 2, 4)) \
#             .reshape((-1, PATCH_SIZE, PATCH_SIZE))
        
        # Use mean as label
#         Y = Y.reshape((-1, PATCH_SIZE * PATCH_SIZE)).mean(axis=1)

        # One-hot encoding
        # Y = keras.utils.to_categorical(Y, NUM_CLASSES)
        Y = np.concatenate([1-Y_patches[:, np.newaxis], Y_patches[:, np.newaxis]], axis=1)
    
    return X_patches, Y
    
# Train/validate split
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.1, random_state=RANDOM_SEED)

X_train, Y_train = prep_data(X_train, Y_train, IMG_SIZE, True)
X_valid, Y_valid = prep_data(X_valid, Y_valid, IMG_SIZE, True)
X_test, _ = prep_data(X_test, None, TEST_IMG_SIZE, False)

100%|██████████| 50/50 [00:00<00:00, 81.29it/s]


In [7]:
dg = ImageDataGenerator(
    channel_shift_range=.2,
    horizontal_flip=True,
    vertical_flip=True,
    data_format='channels_last'
)
dg.fit(X_train)

In [4]:
# Class weights for imbalanced data
num_ones = np.count_nonzero(Y_train[:, 1] > .5)
class_weights = np.array([
    num_ones,
    Y_train.shape[0] - num_ones,
]) / Y_train.shape[0]

In [5]:
model = Sequential()

# 32*32*3
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# 16*16*64
model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# 8*8*128
model.add(Conv2D(256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# 4*4*256
model.add(Conv2D(512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# 2*2*512
model.add(Flatten())
# 2048
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 2048
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
# 512
model.add(Dense(NUM_CLASSES, activation='softmax'))

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    p, r = precision(y_true, y_pred), recall(y_true, y_pred)
    return 2 * p * r / (p + r)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy', f1])

# Choose filename for saved models here
checkpoint = ModelCheckpoint('model2.{epoch:02d}-{val_acc:.4f}.hdf5', 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# Train model
# model.fit(X_train, Y_train, 
#           batch_size=BATCH_SIZE,
#           epochs=30,
#           verbose=1,
#           shuffle=True,
#           class_weight=class_weights,
#           validation_data=(X_valid, Y_valid),
#           callbacks=[checkpoint])

model.fit_generator(dg.flow(X_train, Y_train, batch_size=BATCH_SIZE, shuffle=True), 
          steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
          epochs=30,
          verbose=1,
          class_weight=class_weights,
          validation_data=(X_valid, Y_valid),
          callbacks=[checkpoint])

Epoch 1/30
439/439 [==============================] - 90s - loss: 0.3942 - acc: 0.8263 - f1: 0.7704 - val_loss: 0.3550 - val_acc: 0.8479 - val_f1: 0.8051
Epoch 2/30
439/439 [==============================] - 89s - loss: 0.3307 - acc: 0.8585 - f1: 0.7948 - val_loss: 0.2994 - val_acc: 0.8794 - val_f1: 0.8192
Epoch 3/30
439/439 [==============================] - 88s - loss: 0.3084 - acc: 0.8723 - f1: 0.8088 - val_loss: 0.2792 - val_acc: 0.8756 - val_f1: 0.8341
Epoch 4/30
439/439 [==============================] - 89s - loss: 0.2931 - acc: 0.8817 - f1: 0.8183 - val_loss: 0.2547 - val_acc: 0.8938 - val_f1: 0.8427
Epoch 5/30
439/439 [==============================] - 88s - loss: 0.2791 - acc: 0.8908 - f1: 0.8266 - val_loss: 0.2453 - val_acc: 0.9024 - val_f1: 0.8489
Epoch 6/30
439/439 [==============================] - 89s - loss: 0.2680 - acc: 0.8973 - f1: 0.8322 - val_loss: 0.2558 - val_acc: 0.9044 - val_f1: 0.8424
Epoch 7/30
439/439 [==============================] - 89s - loss: 0.2582 - a

In [6]:
# Restore model
model.load_weights("model2.26-0.9276.hdf5")

In [7]:
# Do prediction
Y_test = model.predict_classes(X_test)

patches_per_axis = TEST_IMG_SIZE // PATCH_SIZE
Y_test = Y_test.reshape((-1, patches_per_axis, patches_per_axis))

# Save predictions
f = open('prediction.csv', 'w')
print('id,prediction', file=f)
for i in range(1, NUM_TEST_FILES+1):
    for x in range(0, patches_per_axis):
        for y in range(0, patches_per_axis):
            print('%.3d_%d_%d,%d' % (i, x * PATCH_SIZE, y * PATCH_SIZE, Y_test[i-1, x, y]), file=f)
f.close()

72160/72200 [============================>.] - ETA: 0s   

In [10]:
X_train[0].min()

0.0